In [7]:
import pandas as pd

# Load the dataset
file_path = 'Weather Training Data.csv'
# Loading the dataset again to work with the original 'Location' values (string names)
weather_data_original = pd.read_csv(file_path)

# Filtering the dataset to focus only on observations from 'Albury'
weather_data_albury = weather_data_original[weather_data_original['Location'] == 'Albury']

weather_data_albury.head()


# # Display the first few rows of the dataset and its summary
# weather_data.head(), weather_data.describe(), weather_data.info()


,row ID,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Row0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0
1,Row1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0
2,Row2,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0
3,Row3,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,0
4,Row4,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,0


In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np

# Handling missing values
# For numeric columns, we'll impute missing values with the mean of the column
numeric_columns = weather_data_albury.select_dtypes(include=[np.number]).columns
imputer_numeric = SimpleImputer(strategy='mean')
weather_data_albury[numeric_columns] = imputer_numeric.fit_transform(weather_data_albury[numeric_columns])

# For categorical columns, we'll impute missing values with the mode (most frequent value) of the column
categorical_columns = weather_data_albury.select_dtypes(include=['object']).columns
imputer_categorical = SimpleImputer(strategy='most_frequent')
weather_data_albury[categorical_columns] = imputer_categorical.fit_transform(weather_data_albury[categorical_columns])

# Encoding categorical variables
label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    weather_data_albury[column] = label_encoders[column].fit_transform(weather_data_albury[column])

# Normalizing the data
scaler = StandardScaler()
weather_data_albury[numeric_columns] = scaler.fit_transform(weather_data_albury[numeric_columns])

# # Check the preprocessed data
# weather_data.head(), weather_data.describe()


/Users/xinranli/Library/Python/3.9/lib/python/site-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: ['Evaporation' 'Sunshine']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


ValueError: Columns must be same length as key

In [3]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into features (X) and target variable (y)
X = weather_data_albury.drop(columns=['RainTomorrow'])
y = weather_data_albury['RainTomorrow']

# Splitting the data into training, validation, and test sets
# Using 70% of data for training, 15% for validation, and 15% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

# Sizes of each dataset
sizes = {
    "Training set size": X_train.shape[0],
    "Validation set size": X_val.shape[0],
    "Test set size": X_test.shape[0]
}

sizes


{'Training set size': 69661,
 'Validation set size': 14927,
 'Test set size': 14928}

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Initialize Logistic Regression model
logreg = LogisticRegression(solver='liblinear', random_state=42)

# Parameters for tuning
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100], # Regularization parameter
    'penalty': ['l1', 'l2'] # Norm used in the penalization
}

# Grid search with cross-validation for parameter tuning
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_val, y_val)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the model with the best parameters on the full training set
logreg_optimized = LogisticRegression(**best_params, solver='liblinear', random_state=42)
logreg_optimized.fit(X_train, y_train)

# Predict on the test set
y_pred = logreg_optimized.predict(X_test)

# Evaluate the optimized model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Printing the best parameters found by Grid Search
print("Best Parameters from Grid Search:", best_params)

# Printing the best score achieved on the validation set during Grid Search
print("Best Score from Grid Search on Validation Set:", best_score)

# Printing the accuracy of the optimized model on the test set
print("Accuracy of Optimized Model on Test Set:", accuracy)

# Printing the detailed classification report for the test set predictions
print("Classification Report for Test Set:")
print(classification_rep)



Best Parameters from Grid Search: {'C': 1, 'penalty': 'l1'}
Best Score from Grid Search on Validation Set: 0.842030559136383
Accuracy of Optimized Model on Test Set: 0.8413049303322615
Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90     11493
           1       0.73      0.49      0.59      3435

    accuracy                           0.84     14928
   macro avg       0.80      0.72      0.74     14928
weighted avg       0.83      0.84      0.83     14928



In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Initialize the Support Vector Machine classifier
svm_classifier = SVC(random_state=42)

# Parameters for tuning
param_grid_svm = {
    'C': [0.1, 1, 10],  # Regularization parameter
    # 'kernel': ['rbf', 'poly'],  # Kernel types
    'kernel': ['rbf'],  # Kernel types
    'gamma': ['scale', 'auto']  # Kernel coefficient
}

# Grid search with cross-validation for parameter tuning
grid_search_svm = GridSearchCV(svm_classifier, param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm.fit(X_val, y_val)

# Best parameters and best score
best_params_svm = grid_search_svm.best_params_
best_score_svm = grid_search_svm.best_score_

# Train the model with the best parameters on the full training set
svm_optimized = SVC(**best_params_svm, random_state=42)
svm_optimized.fit(X_train, y_train)

# Predict on the test set
y_pred_svm = svm_optimized.predict(X_test)

# Evaluate the optimized model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

# Printing the best parameters found by Grid Search
print("Best Parameters from Grid Search for SVM:", best_params_svm)

# Printing the best score achieved on the validation set during Grid Search
print("Best Score from Grid Search on Validation Set for SVM:", best_score_svm)

# Printing the accuracy of the optimized SVM model on the test set
print("Accuracy of Optimized SVM Model on Test Set:", accuracy_svm)

# Printing the detailed classification report for the test set predictions by the SVM model
print("Classification Report for Test Set using SVM:")
print(classification_rep_svm)



Best Parameters from Grid Search for SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Best Score from Grid Search on Validation Set for SVM: 0.7763113850116848
Accuracy of Optimized SVM Model on Test Set: 0.769895498392283
Classification Report for Test Set using SVM:
              precision    recall  f1-score   support

           0       0.77      1.00      0.87     11493
           1       0.00      0.00      0.00      3435

    accuracy                           0.77     14928
   macro avg       0.38      0.50      0.43     14928
weighted avg       0.59      0.77      0.67     14928



/Users/xinranli/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinranli/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinranli/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
